In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000), dtype=np.float64) #100个agent的资源量 定义成一个矩阵
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户资源量的总和
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户成本的总和
iteration_update = 1 #迭代步长 
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(54,54))# 网络的度矩阵
L=np.zeros(shape=(54,54))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数b
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数c
agent_gradient=np.zeros(shape=(54,2000), dtype=np.float64)  
bias=np.loadtxt("2000_Gaussian_distribution_(80,10).txt")

In [3]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,54):
    d=0
    for j in range(0,54):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54
    
#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算 随机选择一个用户作为biased 用户
for i in range (0,54):
    if i==48:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]+bias[0]
    else:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]
        

#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum

In [4]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD_Gaussian_bias(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代  假设部分用户因为设备损坏 导致梯度的计算有误差     
        for i in range (0,54):
            weight_gradient_sum = np.float64(0)
            for j in range (0,54):        
                weight_gradient_sum=weight_gradient_sum+W[i][j]*agent_gradient[j][k-1]
            #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
            #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)
        
        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_c_values[i]*(agent_resource_quantity[i][k])*(agent_resource_quantity[i][k])+generator_b_values[i]*agent_resource_quantity[i][k]+generator_a_values[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,54): 
            if i==48:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]+bias[k]
            else:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%20==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
IWDGD_Gaussian_bias(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias)

****************************IWDGD算法+计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.111111111

49 次迭代中的结果：
agent 1 在 49 次迭代中的资源申请量为: 49.283295057924214
agent 2 在 49 次迭代中的资源申请量为: 62.04368420228544
agent 3 在 49 次迭代中的资源申请量为: 44.58771471281181
agent 4 在 49 次迭代中的资源申请量为: 65.62724780125652
agent 5 在 49 次迭代中的资源申请量为: 66.42724780125653
agent 6 在 49 次迭代中的资源申请量为: 73.28432575353689
agent 7 在 49 次迭代中的资源申请量为: 40.158596569341825
agent 8 在 49 次迭代中的资源申请量为: 38.37162004091118
agent 9 在 49 次迭代中的资源申请量为: 43.887714712811814
agent 10 在 49 次迭代中的资源申请量为: 56.78540721630746
agent 11 在 49 次迭代中的资源申请量为: 46.31226508514472
agent 12 在 49 次迭代中的资源申请量为: 50.18329505792423
agent 13 在 49 次迭代中的资源申请量为: 66.92724780125653
agent 14 在 49 次迭代中的资源申请量为: 56.18540721630747
agent 15 在 49 次迭代中的资源申请量为: 44.28771471281181
agent 16 在 49 次迭代中的资源申请量为: 44.487714712811815
agent 17 在 49 次迭代中的资源申请量为: 45.71226508514473
agent 18 在 49 次迭代中的资源申请量为: 47.31226508514472
agent 19 在 49 次迭代中的资源申请量为: 42.05174238074574
agent 20 在 49 次迭代中的资源申请量为: 56.685407216307475
agent 21 在 49 次迭代中的资源申请量为: 39.45859656934183
agent 22 在 49 次迭代中的资源申请量为: 39.75859656934183
ag

109 次迭代中的结果：
agent 1 在 109 次迭代中的资源申请量为: 50.302697420703396
agent 2 在 109 次迭代中的资源申请量为: 63.307426685018164
agent 3 在 109 次迭代中的资源申请量为: 45.50572551206322
agent 4 在 109 次迭代中的资源申请量为: 67.0065877570032
agent 5 在 109 次迭代中的资源申请量为: 67.80658775700321
agent 6 在 109 次迭代中的资源申请量为: 74.80233804247618
agent 7 在 109 次迭代中的资源申请量为: 41.010157227136375
agent 8 在 109 次迭代中的资源申请量为: 39.200626821442434
agent 9 在 109 次迭代中的资源申请量为: 44.805725512063226
agent 10 在 109 次迭代中的资源申请量为: 57.95260139820834
agent 11 在 109 次迭代中的资源申请量为: 47.27600433760238
agent 12 在 109 次迭代中的资源申请量为: 51.202697420703394
agent 13 在 109 次迭代中的资源申请量为: 68.30658775700321
agent 14 在 109 次迭代中的资源申请量为: 57.352601398208336
agent 15 在 109 次迭代中的资源申请量为: 45.20572551206321
agent 16 在 109 次迭代中的资源申请量为: 45.40572551206322
agent 17 在 109 次迭代中的资源申请量为: 46.676004337602386
agent 18 在 109 次迭代中的资源申请量为: 48.27600433760238
agent 19 在 109 次迭代中的资源申请量为: 42.93269866275382
agent 20 在 109 次迭代中的资源申请量为: 57.85260139820834
agent 21 在 109 次迭代中的资源申请量为: 40.31015722713638
agent 22 在 109 次迭代中的资源申

179 次迭代中的结果：
agent 1 在 179 次迭代中的资源申请量为: 48.991427133176714
agent 2 在 179 次迭代中的资源申请量为: 61.79947297870649
agent 3 在 179 次迭代中的资源申请量为: 44.28459904825913
agent 4 在 179 次迭代中的资源申请量为: 65.40856257498274
agent 5 在 179 次迭代中的资源申请量为: 66.20856257498274
agent 6 在 179 次迭代中的资源申请量为: 73.09683798383725
agent 7 在 179 次迭代中的资源申请量为: 39.858893820783855
agent 8 在 179 次迭代中的资源申请量为: 38.079015411228056
agent 9 在 179 次迭代中的资源申请量为: 43.58459904825912
agent 10 在 179 次迭代中的资源申请量为: 56.52078852043177
agent 11 在 179 次迭代中的资源申请量为: 46.01288596878661
agent 12 在 179 次迭代中的资源申请量为: 49.89142713317672
agent 13 在 179 次迭代中的资源申请量为: 66.70856257498274
agent 14 在 179 次迭代中的资源申请量为: 55.92078852043177
agent 15 在 179 次迭代中的资源申请量为: 43.98459904825912
agent 16 在 179 次迭代中的资源申请量为: 44.18459904825912
agent 17 在 179 次迭代中的资源申请量为: 45.41288596878662
agent 18 在 179 次迭代中的资源申请量为: 47.01288596878662
agent 19 在 179 次迭代中的资源申请量为: 41.74853789350772
agent 20 在 179 次迭代中的资源申请量为: 56.42078852043177
agent 21 在 179 次迭代中的资源申请量为: 39.15889382078385
agent 22 在 179 次迭代中的资源申请量为:

agent 25 在 219 次迭代中的资源申请量为: 45.535910357540594
agent 26 在 219 次迭代中的资源申请量为: 40.08331363023284
agent 27 在 219 次迭代中的资源申请量为: 66.05921571146459
agent 28 在 219 次迭代中的资源申请量为: 51.74680788096627
agent 29 在 219 次迭代中的资源申请量为: 41.14033347340971
agent 30 在 219 次迭代中的资源申请量为: 39.483313630232836
agent 31 在 219 次迭代中的资源申请量为: 60.88227309758428
agent 32 在 219 次迭代中的资源申请量为: 55.93760927464286
agent 33 在 219 次迭代中的资源申请量为: 41.94033347340971
agent 34 在 219 次迭代中的资源申请量为: 52.646807880966286
agent 35 在 219 次迭代中的资源申请量为: 39.183313630232824
agent 36 在 219 次迭代中的资源申请量为: 43.14250052467378
agent 37 在 219 次迭代中的资源申请量为: 52.94680788096628
agent 38 在 219 次迭代中的资源申请量为: 40.640333473409704
agent 39 在 219 次迭代中的资源申请量为: 43.64250052467377
agent 40 在 219 次迭代中的资源申请量为: 50.47897616827034
agent 41 在 219 次迭代中的资源申请量为: 66.75921571146458
agent 42 在 219 次迭代中的资源申请量为: 42.84250052467377
agent 43 在 219 次迭代中的资源申请量为: 52.14680788096627
agent 44 在 219 次迭代中的资源申请量为: 53.346807880966274
agent 45 在 219 次迭代中的资源申请量为: 48.578976168270344
agent 46 在 219 次迭代中的资源申请量为:

...

1769 次迭代中的结果：
agent 1 在 1769 次迭代中的资源申请量为: 49.32059605351771
agent 2 在 1769 次迭代中的资源申请量为: 62.02652380860194
agent 3 在 1769 次迭代中的资源申请量为: 44.663589256939325
agent 4 在 1769 次迭代中的资源申请量为: 65.59331191433918
agent 5 在 1769 次迭代中的资源申请量为: 66.3933119143392
agent 6 在 1769 次迭代中的资源申请量为: 73.23450062566825
agent 7 在 1769 次迭代中的资源申请量为: 40.27469085283267
agent 8 在 1769 次迭代中的资源申请量为: 38.50882911058104
agent 9 在 1769 次迭代中的资源申请量为: 43.963589256939315
agent 10 在 1769 次迭代中的资源申请量为: 56.7858027800722
agent 11 在 1769 次迭代中的资源申请量为: 46.36867499580262
agent 12 在 1769 次迭代中的资源申请量为: 50.2205960535177
agent 13 在 1769 次迭代中的资源申请量为: 66.89331191433918
agent 14 在 1769 次迭代中的资源申请量为: 56.1858027800722
agent 15 在 1769 次迭代中的资源申请量为: 44.36358925693931
agent 16 在 1769 次迭代中的资源申请量为: 44.563589256939316
agent 17 在 1769 次迭代中的资源申请量为: 45.76867499580263
agent 18 在 1769 次迭代中的资源申请量为: 47.36867499580263
agent 19 在 1769 次迭代中的资源申请量为: 42.147469314615854
agent 20 在 1769 次迭代中的资源申请量为: 56.685802780072194
agent 21 在 1769 次迭代中的资源申请量为: 39.57469085283265
agent 2

1819 次迭代中的结果：
agent 1 在 1819 次迭代中的资源申请量为: 49.32665160262157
agent 2 在 1819 次迭代中的资源申请量为: 62.018982897810545
agent 3 在 1819 次迭代中的资源申请量为: 44.68471333308611
agent 4 在 1819 次迭代中的资源申请量为: 65.58535986303855
agent 5 在 1819 次迭代中的资源申请量为: 66.38535986303856
agent 6 在 1819 次迭代中的资源申请量为: 73.22978444873084
agent 7 在 1819 次迭代中的资源申请量为: 40.31357356288486
agent 8 在 1819 次迭代中的资源申请量为: 38.557155480161484
agent 9 在 1819 次迭代中的资源申请量为: 43.98471333308611
agent 10 在 1819 次迭代中的资源申请量为: 56.78114838548272
agent 11 在 1819 次迭代中的资源申请量为: 46.38183782696543
agent 12 在 1819 次迭代中的资源申请量为: 50.22665160262158
agent 13 在 1819 次迭代中的资源申请量为: 66.88535986303856
agent 14 在 1819 次迭代中的资源申请量为: 56.18114838548272
agent 15 在 1819 次迭代中的资源申请量为: 44.384713333086104
agent 16 在 1819 次迭代中的资源申请量为: 44.584713333086114
agent 17 在 1819 次迭代中的资源申请量为: 45.781837826965436
agent 18 在 1819 次迭代中的资源申请量为: 47.38183782696544
agent 19 在 1819 次迭代中的资源申请量为: 42.17722454465309
agent 20 在 1819 次迭代中的资源申请量为: 56.68114838548273
agent 21 在 1819 次迭代中的资源申请量为: 39.613573562884866
ag

1879 次迭代中的结果：
agent 1 在 1879 次迭代中的资源申请量为: 48.96120466672333
agent 2 在 1879 次迭代中的资源申请量为: 61.69223300566541
agent 3 在 1879 次迭代中的资源申请量为: 44.30524404850957
agent 4 在 1879 次迭代中的资源申请量为: 65.2768218348477
agent 5 在 1879 次迭代中的资源申请量为: 66.0768218348477
agent 6 在 1879 次迭代中的资源申请量为: 72.94200908074104
agent 7 在 1879 次迭代中的资源申请量为: 39.92777693874809
agent 8 在 1879 次迭代中的资源申请量为: 38.170837004184946
agent 9 在 1879 次迭代中的资源申请量为: 43.60524404850956
agent 10 在 1879 次迭代中的资源申请量为: 56.43889318272431
agent 11 在 1879 次迭代中的资源申请量为: 46.008337475451256
agent 12 在 1879 次迭代中的资源申请量为: 49.861204666723324
agent 13 在 1879 次迭代中的资源申请量为: 66.5768218348477
agent 14 在 1879 次迭代中的资源申请量为: 55.838893182724306
agent 15 在 1879 次迭代中的资源申请量为: 44.00524404850956
agent 16 在 1879 次迭代中的资源申请量为: 44.20524404850956
agent 17 在 1879 次迭代中的资源申请量为: 45.408337475451255
agent 18 在 1879 次迭代中的资源申请量为: 47.00833747545125
agent 19 在 1879 次迭代中的资源申请量为: 41.79369590385123
agent 20 在 1879 次迭代中的资源申请量为: 56.338893182724306
agent 21 在 1879 次迭代中的资源申请量为: 39.227776938748086
agen

1929 次迭代中的结果：
agent 1 在 1929 次迭代中的资源申请量为: 48.784288789745226
agent 2 在 1929 次迭代中的资源申请量为: 61.551751841424085
agent 3 在 1929 次迭代中的资源申请量为: 44.102214904263136
agent 4 在 1929 次迭代中的资源申请量为: 65.14454913089735
agent 5 在 1929 次迭代中的资源申请量为: 65.94454913089736
agent 6 在 1929 次迭代中的资源申请量为: 72.81430515987314
agent 7 在 1929 次迭代中的资源申请量为: 39.70210906635314
agent 8 在 1929 次迭代中的资源申请量为: 37.93560849480353
agent 9 在 1929 次迭代中的资源申请量为: 43.40221490426314
agent 10 在 1929 次迭代中的资源申请量为: 56.28762759307395
agent 11 在 1929 次迭代中的资源申请量为: 45.8181148614746
agent 12 在 1929 次迭代中的资源申请量为: 49.68428878974523
agent 13 在 1929 次迭代中的资源申请量为: 66.44454913089736
agent 14 在 1929 次迭代中的资源申请量为: 55.687627593073955
agent 15 在 1929 次迭代中的资源申请量为: 43.80221490426314
agent 16 在 1929 次迭代中的资源申请量为: 44.00221490426314
agent 17 在 1929 次迭代中的资源申请量为: 45.218114861474604
agent 18 在 1929 次迭代中的资源申请量为: 46.818114861474605
agent 19 在 1929 次迭代中的资源申请量为: 41.578774621131316
agent 20 在 1929 次迭代中的资源申请量为: 56.18762759307397
agent 21 在 1929 次迭代中的资源申请量为: 39.00210906635315
ag

1989 次迭代中的结果：
agent 1 在 1989 次迭代中的资源申请量为: 49.1741845818357
agent 2 在 1989 次迭代中的资源申请量为: 61.89972893098601
agent 3 在 1989 次迭代中的资源申请量为: 44.526185722537335
agent 4 在 1989 次迭代中的资源申请量为: 65.48547386635371
agent 5 在 1989 次迭代中的资源申请量为: 66.28547386635373
agent 6 在 1989 次迭代中的资源申请量为: 73.1535300611212
agent 7 在 1989 次迭代中的资源申请量为: 40.156801515194765
agent 8 在 1989 次迭代中的资源申请量为: 38.40299895352136
agent 9 在 1989 次迭代中的资源申请量为: 43.826185722537346
agent 10 在 1989 次迭代中的资源申请量为: 56.646901982994045
agent 11 在 1989 次迭代中的资源申请量为: 46.22510106455892
agent 12 在 1989 次迭代中的资源申请量为: 50.07418458183571
agent 13 在 1989 次迭代中的资源申请量为: 66.78547386635373
agent 14 在 1989 次迭代中的资源申请量为: 56.046901982994044
agent 15 在 1989 次迭代中的资源申请量为: 44.22618572253734
agent 16 在 1989 次迭代中的资源申请量为: 44.42618572253734
agent 17 在 1989 次迭代中的资源申请量为: 45.625101064558926
agent 18 在 1989 次迭代中的资源申请量为: 47.22510106455892
agent 19 在 1989 次迭代中的资源申请量为: 42.01884927783489
agent 20 在 1989 次迭代中的资源申请量为: 56.546901982994044
agent 21 在 1989 次迭代中的资源申请量为: 39.45680151519476
age

In [6]:
np.savetxt('../data/03-agent_resource_quantity_small_value_Gaussian_bias(80,10).txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/03-sum_cost_small_value_Gaussian_bias(80,10).txt',sum_cost,fmt='%0.8f')#用户成本总和 
np.savetxt('../data/03-sum_resource_small_value_Gaussian_bias(80,10).txt',sum_resource,fmt='%0.8f')#用户资源总和 
np.savetxt('../data/03-agent_gradient_small_value_Gaussian_bias(80,10).txt',agent_gradient,fmt='%0.8f')#用户迭代梯度 